In [1]:
import sys
paths_to_add = ["/homes/vk223/ProjectFlood", "/home/vkhandekar/project_flood"]
for p in paths_to_add:
    if p not in sys.path:
        sys.path.append(p)
print(sys.path)

from model_runs.model_run_helpers import *
from model_runs.distributed_gpu_helpers import *
from visualisations.visualisation_helpers import *
import os
print(os.getcwd())


['/usr/lib/python310.zip', '/usr/lib/python3.10', '/usr/lib/python3.10/lib-dynload', '', '/vol/bitbucket/vk223/project_flood/projectfloodvenv/lib/python3.10/site-packages', '/homes/vk223/ProjectFlood', '/home/vkhandekar/project_flood']
/homes/vk223/ProjectFlood/evaluation


In [2]:
from evaluation.model_evaluation_helpers import *
os.environ["PROJECT_FLOOD_DATA"] = "/homes/vk223/ProjectFlood/static/imperial_data_paths.json"
os.environ["PROJECT_FLOOD_CORE_PATHS"] = "/homes/vk223/ProjectFlood/static/imperial_core_paths.json"
print(os.environ["PROJECT_FLOOD_DATA"])


/homes/vk223/ProjectFlood/static/imperial_data_paths.json


In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
torch.manual_seed(42)
# Load newly trained best, final models
final_sep_branch_model, _, _, final_sep_branch_params = load_checkpoint("/homes/vk223/ProjectFlood/models/saved_models/ConvLSTMMerged_epochs500_batchsize16_lr0p001_precedingrainfall1_dropout0p3_outputchannels8_convblocklayers2_convLSTMlayers2_optimRMSprop_criterionBCELoss_transformsFalse_res256_20240824_500.pt")
final_merged_model, _, _, final_merged_params = load_checkpoint("/homes/vk223/ProjectFlood/models/saved_models/ConvLSTMSeparateBranches_epochs500_batchsize32_lr0p001_precedingrainfall1_dropout0p5_outputchannels8_convblocklayers2_convLSTMlayers1_optimRMSprop_criterionBCELoss_transformsFalse_res256_20240824_500.pt")

dict_keys(['epochs', 'batchsize', 'lr', 'precedingrainfall', 'dropout', 'outputchannels', 'convblocklayers', 'convLSTMlayers', 'optim', 'criterion', 'transforms', 'res'])
dict_keys(['epochs', 'batchsize', 'lr', 'precedingrainfall', 'dropout', 'outputchannels', 'convblocklayers', 'convLSTMlayers', 'optim', 'criterion', 'transforms', 'res'])


/homes/vk223/ProjectFlood/evaluation/model_evaluation_helpers.py:16: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(filepath)
/vol/bitbucket/vk223/pro

In [4]:
sep_branch_test_dataloader = get_dataloader("test_labels_path", resolution=256, preceding_rainfall_days=final_sep_branch_params['precedingrainfall'], forecast_rainfall_days=1, 
                                                transform=None, batch_size=16, shuffle=False, num_workers=4)
    
merged_test_dataloader = get_dataloader("test_labels_path", resolution=256, preceding_rainfall_days=final_merged_params['precedingrainfall'], forecast_rainfall_days=1, 
                                            transform=None, batch_size=16, shuffle=False, num_workers=4)

In [5]:
from visualisations.visualisation_helpers import *
# Plot image
with torch.no_grad():
    size = 3

    sep_branch_flooded_images = []
    sep_branch_non_flooded_images = []
    for inputs, targets, flooded in sep_branch_test_dataloader:
        sep_branch_outputs = final_sep_branch_model(inputs)
        for i in range(len(flooded)):
            if flooded[i] == 1 and len(sep_branch_flooded_images) < size:
                sep_branch_flooded_images.append((sep_branch_outputs[i], targets[i], flooded[i]))
            elif flooded[i] == 0 and len(sep_branch_non_flooded_images) < size:
                sep_branch_non_flooded_images.append((sep_branch_outputs[i], targets[i], flooded[i]))
            # Stop if we've collected 4 images in each category
            if len(sep_branch_flooded_images) >= size and len(sep_branch_non_flooded_images) >= size:
                break
        if len(sep_branch_flooded_images) >= 4 and len(sep_branch_non_flooded_images) >= 4:
            break

    merged_flooded_images = []
    merged_non_flooded_images = []
    for inputs, targets, flooded in merged_test_dataloader:
        merged_outputs = final_merged_model(inputs)
        for i in range(len(flooded)):
            if flooded[i] == 1 and len(merged_flooded_images) < size:
                merged_flooded_images.append((merged_outputs[i], targets[i], flooded[i]))
            elif flooded[i] == 0 and len(merged_non_flooded_images) < size:
                merged_non_flooded_images.append((merged_outputs[i], targets[i], flooded[i]))
            # Stop if we've collected 4 images in each category
            if len(merged_flooded_images) >= size and len(merged_non_flooded_images) >= size:
                break
        if len(merged_flooded_images) >= 4 and len(merged_non_flooded_images) >= 4:
            break
    
    with open(os.environ["PROJECT_FLOOD_CORE_PATHS"]) as core_config_file:
        core_config = json.load(core_config_file)
    dimension_string = core_config[f"rainfall_reprojection_master_256"]
    match = re.search(r'_(\d+)_(\d+)\.tif$', dimension_string)
    new_dimension_right, new_dimension_bottom = int(match.group(1)), int(match.group(2))


    selected_sep_branch_outputs = [img[0] for img in sep_branch_flooded_images + sep_branch_non_flooded_images]
    selected_sep_branch_outputs = [i[:new_dimension_bottom, :new_dimension_right] for i in selected_sep_branch_outputs] #crop


    selected_merged_outputs = [img[0] for img in merged_flooded_images + merged_non_flooded_images]
    selected_merged_outputs = [i[:new_dimension_bottom, :new_dimension_right] for i in selected_merged_outputs] #crop

    model_names = ['Branched', 'Merged']
    selected_model_outputs = [selected_sep_branch_outputs, selected_merged_outputs]

    selected_targets = [img[1] for img in sep_branch_flooded_images + sep_branch_non_flooded_images]
    selected_targets = [i[:new_dimension_bottom, :new_dimension_right] for i in selected_targets] #crop


    selected_targets_flooded = [img[2] for img in sep_branch_flooded_images + sep_branch_non_flooded_images] #boolean for flooded or not

    # Do plotting
    with open(os.environ["PROJECT_FLOOD_DATA"]) as data_config_file:
        data_config = json.load(data_config_file)
    plot_filename = os.path.join(data_config["model_results_path"], "final_plots.png")
    plot_final_model_output_vs_label(model_names, selected_model_outputs, selected_targets, selected_targets_flooded, plot_filename)